In [1]:
from keras.models import Sequential
from keras.layers import Dense
import pandas as pd


In [2]:
df_final = pd.read_csv("/Users/shafin/documents/Pitch/data-for-models/df_final.csv") # dataframe of all 3 seasons
df_final_2018 = pd.read_csv("/Users/shafin/documents/Pitch/data-for-models/df_final_2018.csv") # dataframe for 2018 season
df_final_100 = pd.read_csv("/Users/shafin/documents/Pitch/data-for-models/df_final_100.csv") # dataframe for 100 games

In [ ]:
import seaborn as sns
import matplotlib as plt
corr = df_2018.corr()
sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values)

In [3]:
# One-hot encode the data using pandas get_dummies
df_final_100 = pd.get_dummies(df_final_100)
df_final_100.head(5)

,s_count,b_count,pitch_num,on_1b,on_2b,on_3b,pitcher_id,fastball,stand_L,stand_R,p_throws_L,p_throws_R,last_ff_not_CH,last_ff_not_CU/SL,last_ff_not_FF,last_ff_not_OTHER
0,0,0,1,0,0,0,608665,0,0,1,0,1,0,0,0,0
1,1,0,2,0,0,0,608665,0,0,1,0,1,0,0,0,1
2,1,1,3,0,0,0,608665,0,0,1,0,1,0,0,0,1
3,2,1,4,0,0,0,608665,0,0,1,0,1,0,0,0,1
4,2,1,5,0,0,0,608665,0,0,1,0,1,0,1,0,0


In [4]:
from sklearn.model_selection import train_test_split
X=df_final_100.drop('fastball', axis=1)  # Features
y=df_final_100.fastball  # Labels
# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test

In [ ]:
# creating the model
model = Sequential()
model.add(Dense(8, input_shape=(15,), activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# compile the keras model
model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train,epochs=4, batch_size=1, verbose=1)

In [ ]:
y_pred = model.predict_classes(X_test)

In [ ]:
score = model.evaluate(X_test, y_test,verbose=1)
print(score)

In [6]:
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
# Function to create model, required for KerasClassifier
def create_model(optimizer='adam'):
	# create model
	model = Sequential()
	model.add(Dense(12, input_shape=(15,), activation='relu'))
    model.add(Dense(8, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
	return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load dataset
# create model
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
# define the grid search parameters
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(optimizer=optimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

ValueError: in user code:

    C:\Users\Shafin\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    C:\Users\Shafin\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\Shafin\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\Shafin\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\Shafin\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\Shafin\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:789 run_step